In [ ]:
import variational_bayes as vb
import numpy as np
from matplotlib import pyplot as plt, rcParams
import scipy.stats
from tqdm import tqdm_notebook
%matplotlib inline

rcParams['figure.dpi'] = 144
rcParams['scatter.marker'] = '.'

In [ ]:
np.random.seed(3)
num_groups = 3
num_nodes = 50
order = 2
num_steps = 100 * num_nodes * num_nodes

# Generate group sizes and groups
density = np.random.dirichlet(100 * np.ones(num_groups)) # was 10 when things didn't work
z = np.random.choice(num_groups, num_nodes, p=density)
onehot = np.zeros((num_nodes, num_groups))
onehot[np.arange(num_nodes), z] = 1

# Sample noise precisions for all groups
noise_precision = np.random.gamma(5000, size=num_groups)
# noise_precision = 100

# Sample means and precisions of autoregressive coefficients
adjacency_mean = np.random.normal(0, 1e-2, size=(num_groups, num_groups, order))
adjacency_precision = scipy.stats.wishart.rvs(1e5, np.eye(order), size=(num_groups, num_groups))
if adjacency_precision.ndim < 4:
    adjacency_precision = adjacency_precision.reshape((num_groups, num_groups, 1, 1))

# Sample the means and precisions of the bias
bias_mean = np.random.normal(0, 0.1, num_groups)
bias_precision = np.random.gamma(1e4, 1, num_groups)

# Sample the matrix of autoregressive coefficients
cholesky = np.linalg.cholesky(np.linalg.inv(adjacency_precision))
cholesky = cholesky[z[:, None], z[None, :]]
adjacency = adjacency_mean[z[:, None], z[None, :]] + \
    np.einsum('...ij,...j', cholesky, np.random.normal(0, 1, (num_nodes, num_nodes, order)))
    
# Sample the bias
bias = np.random.normal(0, 1, num_nodes) / np.sqrt(bias_precision[z]) + bias_mean[z]

# Construct the coefficients for comparison
coefficients = vb.pack_coefficients(adjacency, bias)
    
# Generate the actual series
series = vb.simulate_series(bias, adjacency, noise_precision[z], num_steps)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.plot(series)
if order == 2:
    ax2.scatter(*adjacency.reshape((-1, 2)).T, 
                c=(z[:, None] * num_groups + z[None, :]).ravel(),
                cmap='Set1')
    ax2.set_aspect('equal')
    for i in range(num_groups):
        for j in range(num_groups):
            ellipse = vb.ellipse_from_cov(
                adjacency_mean[i,j], np.linalg.inv(adjacency_precision[i,j]),
                facecolor='none', edgecolor='k'
            )
            ax2.add_artist(ellipse)
else:
    for i in range(num_groups):
        for j in range(num_groups):
            color = 'C%d' % ((i * num_groups + j) % 10)
            ax2.hist(adjacency[(z==i)[:, None] & (z==j)[None, :]], color=color, normed=True)
            

for i in range(num_groups):
    ax3.hist(bias[z == i], range=(np.min(bias), np.max(bias)))

In [ ]:
np.random.seed(15)
given = {
    'adjacency_precision': adjacency_precision, 
    'adjacency_mean': adjacency_mean,
    'bias_mean': bias_mean,
    'bias_precision': bias_precision,
    'density': density,
    'z': onehot
}
given = {}
model = vb.var_model(series, order, num_groups, given=given, 
                     uniform_ic=True, shared_noise=False)

In [ ]:
elbos, _ = model.update(None, tqdm_notebook, convergence_predicate=vb.ConvergencePredicate(1e-4, 10))
plt.plot(elbos)

In [ ]:
ordering = vb.cluster_order(model['z'].mean, onehot)
plt.imshow(model['z'].mean[np.argsort(z)], aspect='auto')
ordering

In [ ]:
for i in range(order):
    plt.figure()
    x = model['adjacency_mean'].mean[..., i]
    vmax = np.max(np.abs(x))
    plt.imshow(x, vmax=vmax, vmin=-vmax, cmap='coolwarm')
    plt.title(str(i))

In [ ]:
idx = np.argsort(np.argmax(model['z'].mean, axis=1))
map_estimate = vb.VARDistribution.coefficient_mle(series, order)

q = vb.VARAdjacencyDistribution(model['coefficients'])
for i in range(order):
    fig, (ax1, ax2) = plt.subplots(1, 2, True, True)
    x = q.mean[..., i]
    x = x[idx, :][:, idx] 
    vmax = np.max(np.abs(x))
    ax1.imshow(x, vmin=-vmax, vmax=vmax, cmap='coolwarm')
    ax2.imshow(adjacency[..., i][idx][:, idx], vmin=-vmax, vmax=vmax, cmap='coolwarm')
    fig.suptitle(str(i))
    
    for ax in (ax1, ax2):
        for i in np.nonzero(np.diff(z[idx]))[0]:
            ax.axhline(i, color='k')
            ax.axvline(i, color='k')

In [ ]:
vb.plot_residuals(model['noise_precision'], noise_precision[z], marker='.')

In [ ]:
vb.plot_residuals(vb.VARAdjacencyDistribution(model['coefficients']), adjacency)

In [ ]:
plt.figure()
vb.plot_comparison(model['bias_mean'], bias_mean[ordering], marker='.')
plt.title('bias_mean')

In [ ]:
plt.figure()
vb.plot_comparison(model['bias_precision'], bias_precision[ordering], marker='.')
plt.title('bias_precision')

In [ ]:
plt.figure()
vb.plot_comparison(model['adjacency_precision'], adjacency_precision[ordering][:, ordering], marker='.')
plt.title('adjacency_precision')

In [ ]:
plt.figure()
vb.plot_residuals(model['adjacency_mean'], adjacency_mean[ordering][:, ordering], marker='.')
plt.title('adjacency_mean')

plt.figure()
_x, _y = adjacency.reshape((-1, order)).T
pts = plt.scatter(_x, _y, c=(z[:, None] * num_groups + z[None, :]).ravel(),
            cmap='Set1', zorder=9)
_x, _y = model['adjacency_mean'].mean.reshape((-1, 2)).T
_xerr, _yerr = model['adjacency_mean'].std.reshape((-1, 2)).T
plt.errorbar(_x, _y, _yerr, _xerr, color='k', ls='none', zorder=99)
plt.scatter(_x, _y, marker='x', color='k', zorder=99)
plt.gca().set_aspect(1)
plt.title('adjacency_mean')
plt.colorbar(pts)

In [ ]:
plt.figure()
vb.plot_comparison(model['density'], density[ordering], marker='.')
plt.title('density')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, True, True)

if 'coefficients' in model._factors:
    q_adjacency = vb.VARAdjacencyDistribution(model['coefficients'])
else:
    q_adjacency = vb.VARAdjacencyDistribution(coefficients)
_xerr, _yerr = q_adjacency.std.reshape(-1, order).T * 3
_x, _y = q_adjacency.mean.reshape((-1, order)).T
ax1.errorbar(_x, _y, _yerr, _xerr, ls='none', c='k', alpha=.1)
c = (z[:, None] * num_groups + z[None, :]).ravel()
ax1.scatter(_x, _y, c=c, cmap='Set1', zorder=9)
ax1.set_aspect(1)

for i in range(num_groups):
    for j in range(num_groups):
        if 'adjacency_precision' in model._factors:
            cov = np.linalg.inv(model['adjacency_precision'].mean[i, j])
            xy = model['adjacency_mean'].mean[i, j]
        else:
            cov = np.linalg.inv(adjacency_precision[i, j])
            xy = adjacency_mean[i, j]
            
        ellipse = vb.ellipse_from_cov(
            xy,
            cov,
            facecolor='none', edgecolor='k', zorder=10
        )
        ax1.add_artist(ellipse)
        

_x, _y = vb.VARDistribution.coefficient_mle(series, order)[:, 1:].reshape((-1, 2)).T
ax2.scatter(_x, _y, c=c, cmap='Set1', zorder=9)


for i in range(num_groups):
    for j in range(num_groups):
        ellipse = vb.ellipse_from_cov(
            adjacency_mean[i, j], 
            np.linalg.inv(adjacency_precision[i, j]),
            facecolor='none', edgecolor='k', zorder=10
        )
        ax2.add_artist(ellipse)

for ax in (ax1, ax2):
    ax.axvline(0, ls=':')
    ax.axhline(0, ls=':')